In [1]:
#Read the data
import pandas as pd
import numpy as np
import cv2
from PIL import Image

#df= pd.read_csv('driving_log.csv')
#heading = ['center','left','right']
#image_set = []
#measurements  = []

def load_images(df,heading):
    img_paths = df[heading]    
    images = np.array([np.array(Image.open(fname)) for fname in img_paths])   
    return images

#img_paths = df['center']    
#image_set = np.array([np.array(Image.open(fname)) for fname in img_paths])
#measurements = np.array(df['steering'])
#print(img_paths)                        
                        
#aug_images, aug_measures = [] , []

#x_train = np.array(image_set)
#y_train = measurements.astype(float)



In [4]:
import csv
from PIL import Image
samples = []
car_images = []
action_measures = []


with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)


for row in samples[1:]:        
    steering_center = float(row[3])
    throttle = float(row[4])
    brake = float(row[5])
    speed = float(row[6])
    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction
    # read in images from center, left and right cameras
    path = "" # fill in the path to your training IMG directory
    img_center = np.asarray(Image.open(row[0].strip()))
    action_center = [steering_center,throttle,brake,speed]    
    img_left = np.asarray(Image.open(row[1].strip()))
    action_left = [steering_left,throttle,brake,speed]
    #print(row[0].strip())
    img_right = np.asarray(Image.open(row[2].strip()))
    action_right = [steering_center,throttle,brake,speed]
    # add images and angles to data set
    car_images.extend([img_center, img_left, img_right])
    action_measures.extend([action_center,action_left,action_right])        

print(action_measures[0])        



[0.0, 0.0, 0.0, 22.14829]


In [ ]:
def aug_image_set(car_images,action_measures):
    aug_images = []
    aug_measures = []
    for img,measures in zip(car_images, action_measures):
        aug_images.append(img)
        aug_measures.append(measures)    
        aug_images.append(cv2.flip(img,1))
        measures[0] = measures[0]* -1
        aug_measures.append(measures)
    return [aug_images,aug_measures]


car_images , action_measures = aug_image_set(car_images, action_measures)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [ ]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [15]:
#Create model

from keras.models import  Sequential
from keras.layers import  Dense, Flatten, Lambda


#to be defined as nvidia CUDANN
model = Sequential()
model.add(Lambda( lambda x: x/ 255.0-0.5 , input_shape = (160,320,3)))

model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse',optimizer = 'adam')
model.summary()
model.fit(x_train,y_train, validation_split = 0.2 , shuffle='true')

model.save('model.h5')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 153600)        0           lambda_1[0][0]                   
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 1)             153601      flatten_3[0][0]                  
Total params: 153,601
Trainable params: 153,601
Non-trainable params: 0
____________________________________________________________________________________________________
Train on 6428 samples, validate on 1608 samples
Epoch 1/10
6428/6428 [==============================] - 16s - loss: 2.83